In [0]:
!wget https://github.com/cfotache/pytorch_objectdetecttrack

In [0]:
from models import *
from utils import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [0]:
ls

config/            __pycache__/                    sort.py
images/            PyTorch_Object_Detection.ipynb  utils/
models.py          PyTorch_Object_Tracking.ipynb
object_tracker.py  README.md


In [0]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [0]:
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]

In [0]:
videopath = 'video2.mp4'

%pylab inline 
import cv2
from IPython.display import clear_output

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# initialize Sort object and video capture
from sort import *
vid = cv2.VideoCapture(videopath)
frame_width = int(vid.get(3))
frame_height = int(vid.get(4))
mot_tracker = Sort() 
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
# while(vid.isOpened()):
pre_tracker = None
cnt = 0;
for ii in range(400):
    ret, frame = vid.read()
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pilimg = Image.fromarray(frame)
    detections = detect_image(pilimg)
    print(ii)
    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu())
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])
            color = colors[int(obj_id) % len(colors)]
            color = [i * 255 for i in color]
            cls = classes[int(cls_pred)]
            cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
            cv2.rectangle(frame, (x1, y1-35), (x1+len(cls)*19+60, y1), color, -1)
            result = ""
            if pre_tracker is not None:
                for pre_x1, pre_y1, pre_x2, pre_y2, pre_obj_id, pre_cls_pred in pre_tracker:
                    if obj_id == pre_obj_id:
                        pre_y1 = int(((pre_y1 - pad_y // 2) / unpad_h) * img.shape[0])
                        pre_x1 = int(((pre_x1 - pad_x // 2) / unpad_w) * img.shape[1])
                        print("id",obj_id)
                        print("cur",y1)
                        print("pre",pre_y1)
                        print(ii)
                        if y1 < pre_y1:
                            result = "out"
                        else:
                            result = "come"
                cv2.putText(frame, cls + "-" + str(box_h * box_w) + " " + result, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
        pre_tracker = tracked_objects
    out.write(frame)
vid.release()
out.release()

Populating the interactive namespace from numpy and matplotlib
0
1
id 2522.0
cur 227
pre 227
1
id 2521.0
cur 222
pre 222
1
id 2520.0
cur 221
pre 221
1
id 2519.0
cur 238
pre 238
1
id 2518.0
cur 266
pre 266
1
id 2517.0
cur 263
pre 263
1
id 2516.0
cur 278
pre 278
1
id 2515.0
cur 269
pre 269
1
id 2514.0
cur 353
pre 352
1
id 2513.0
cur 350
pre 350
1
id 2512.0
cur 344
pre 343
1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


2
id 2522.0
cur 225
pre 227
2
id 2520.0
cur 220
pre 221
2
id 2519.0
cur 238
pre 238
2
id 2518.0
cur 263
pre 266
2
id 2517.0
cur 264
pre 263
2
id 2516.0
cur 280
pre 278
2
id 2515.0
cur 268
pre 269
2
id 2514.0
cur 349
pre 353
2
id 2513.0
cur 346
pre 350
2
id 2512.0
cur 342
pre 344
2
3
id 2522.0
cur 225
pre 225
3
id 2520.0
cur 220
pre 220
3
id 2519.0
cur 238
pre 238
3
id 2518.0
cur 261
pre 263
3
id 2517.0
cur 265
pre 264
3
id 2516.0
cur 281
pre 280
3
id 2515.0
cur 265
pre 268
3
id 2514.0
cur 345
pre 349
3
id 2513.0
cur 343
pre 346
3
id 2512.0
cur 336
pre 342
3
4
id 2522.0
cur 225
pre 225
4
id 2520.0
cur 220
pre 220
4
id 2519.0
cur 237
pre 238
4
id 2518.0
cur 260
pre 261
4
id 2517.0
cur 268
pre 265
4
id 2516.0
cur 287
pre 281
4
id 2515.0
cur 263
pre 265
4
id 2514.0
cur 342
pre 345
4
id 2513.0
cur 341
pre 343
4
id 2512.0
cur 333
pre 336
4
5
id 2522.0
cur 225
pre 225
5
id 2520.0
cur 220
pre 220
5
id 2519.0
cur 237
pre 237
5
id 2518.0
cur 258
pre 260
5
id 2517.0
cur 272
pre 268
5
id 2516.0
cu

In [0]:
pip install filterpy

     |████████████████████████████████| 184kB 3.4MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp36-none-any.whl size=110453 sha256=520ba4b1b27fec6ae6e04a9032399156fd69cd20dfb31632ea178133bea2f879
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy
